# Applied Data Analysis: Movies
This project analyzes the key factors influencing box-office success for films released between 1914 and 2016, seeking to understand how audience preferences and industry trends have evolved over time.

We will work with two datasets:
- CMU Movie Summary Corpus
- IMDb dataset

## Loading the Datasets

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
import os
import sys

from importlib import reload
import src.utils.utils
reload(src.utils.utils)
from src.utils.utils import top_n_by_interval
from src.utils.utils import top_n_total_revenue

In [2]:
data_folder = './data/'
pickle_folder = data_folder + 'pickle/'
imdb_folder = data_folder + 'IMDB/'
cmu_folder = data_folder + 'CMU/'

In [3]:
## Load the data

with open(pickle_folder + 'movies.p', 'rb') as f:
    movie_data = pickle.load(f)

with open(pickle_folder + 'characters.p', 'rb') as f:
    character_data = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './data/pickle/movies.p'

In [ ]:
movie_data

In [ ]:
character_data

In [ ]:
# Copy the data to not lose the original one and drop the rows with missing values in the 'Movie release date' column
# since we are studying the period 1910-2016

with open(pickle_folder + 'movies_averageRating.p', 'rb') as f:
    md = pickle.load(f)

# 1. Data Analysis

Firstly, we perform an exploratory data analysis. We focus on a variety of characteristics - such as genre, release time, translations, and lead actors' age and gender - to determine their impact on a film's revenue. By examining five historical periods, we trace how these factors’ influence on box-office success has evolved, reflecting broader societal, political, and historical changes.

## 1.1 Analysis per Year

We start by analyzing the number of films and box-office revenues over the entire study period to understand how the role of movie productions has evolved over time.

### 1.1.1  Number of movies per Year

Firstly, we can observe that the number of movies has increased over the years,suggesting a growth of film production worldwide.

In [ ]:
md_number = md.copy().dropna(subset=['Movie_release_date'])
movies_per_year = md_number['Year'].value_counts().sort_index()

plt.figure(figsize=(6, 4))
plt.plot(movies_per_year.index, movies_per_year.values, linestyle='-', color='teal')
plt.fill_between(movies_per_year.index, movies_per_year.values, color='teal', alpha=0.5)
plt.xlabel("Year")
plt.ylabel("Number of Movies")
plt.title("Evolution of Number of Movies per Year")
plt.show()

### 1.1.2 Box Office Revenue per Year

In this section, the plot shows the evolution of mean box office revenue over the years. The black line represents the mean revenue over time, while the shaded blue area indicates the variability or spread around this mean. 

Initially, from 1915 to around 1950, box office revenue remains relatively low and stable. Starting in the 1950s, however, we observe a gradual upward trend, reflecting increased box office revenue, likely driven by the growing popularity of cinema, advancements in film production, and the post-war economic boom. 

From the 1970s onward, the revenue shows more pronounced growth, with increased variability. Indeed, this period corresponds to changes in distribution methods and the impact of global markets. The trend indicates that while the film industry has become more profitable over time, it has also become more volatile, suggesting some films achieving massive box office success while others underperform.

In [ ]:
## Calculate the average and standard deviation of the box office revenue per year

stats_per_year = md_number.groupby(by='Year')['Movie_box_office_revenue'].agg(
    average='mean',
    std_dev='std'
).dropna()

ax = stats_per_year.average.plot(xlim=(1910,2023),xticks=range(1895,2023,10),title='Evolution of Box Office mean Revenue',ylabel='Box office Revenue (in hundreds of Millions)',color='black')
plt.fill_between(stats_per_year.index, stats_per_year.average - stats_per_year.std_dev,
                 stats_per_year.average + stats_per_year.std_dev, alpha = 0.5, color = 'skyblue')

## 1.2 Analysis per Country

In this section, we analyze the number of movies, box-office revenues, and film ratings by country to understand cultural preferences, evaluate the impact of international markets, detect success disparities, observe the evolution of the global film industry, and identify the factors that contribute to a film's success.

In [ ]:
with open(pickle_folder + 'movies_countries_ratings.p', 'rb') as f:
    md_countries = pickle.load(f)

### 1.2.1 Number of Movies per Country

The United States of America leads by a significant margin, with over 30,000 movies, far surpassing other countries. India follows, but with much lower numbers, while the United Kingdom, France, and Italy are also prominent.

In [ ]:
top_countries = md_countries.value_counts('Countries').head(10)

plt.figure(figsize=(8, 3))
sns.barplot(y=top_countries.index, x=top_countries.values, palette="viridis",hue=top_countries.index)
plt.title('Top 10 Most Frequent Movie Countries')
plt.xlabel('Number of Movies')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

Additionally, we analyze the 10 most frequent countries in film production over a 20-year interval:

In [ ]:
ax = top_n_by_interval('Countries',md_countries)

### 1.2.2 Box Office Revenue and Average Rating per Country

The figure below contains two bar plots: 
- Left Plot (Average IMDb Rating by Country): This plot shows the top 10 countries by average IMDb rating.
- Right Plot (Average Box Office Revenue by Country): This plot shows the top 10 countries by average box office revenue, displayed on a logarithmic scale.

Interestingly, major film-producing countries, like the United States or India, are not present in this top 10 list for box office revenue, while countries like New Zealand, Czech Republic, and the United Arab Emirates appear.

Some factors could explain why some lesser-known film-producing countries appear at the top for average IMDb ratings or box office revenue, rather than countries with traditionally large film industries:
- Small Sample Size: Countries with fewer film productions might have a few well-rated or high-revenue films that skew the averages.
- High-Profile Productions: Some of these countries may not have large film industries, but they could have hosted high-budget international productions that brought in significant revenue or received favorable ratings.
- Cultural Biases in Ratings: IMDb ratings are user-generated, which can introduce cultural biases.

In [ ]:
md_countries = md_countries.dropna(subset=['Movie_box_office_revenue'])

country_revenue = md_countries.groupby('Countries').agg({
    'averageRating': 'mean',
    'numVotes': 'mean',
    'Movie_box_office_revenue': 'mean'
})

# Plot 
fig, axs = plt.subplots(1, 2, figsize=(18, 8))
country_revenue = country_revenue.sort_values(by='averageRating', ascending=False)

# Average imdb rating by Language
sns.barplot(x=country_revenue.head(10).index, y=country_revenue.head(10)['averageRating'], ax=axs[0], palette='viridis', hue=country_revenue.head(10).index)    
axs[0].set_title('Average IMDb Rating by Country')
axs[0].set_xlabel('Country')
axs[0].set_ylabel('Average Rating')
axs[0].tick_params(axis='x', rotation=90)  

country_revenue = country_revenue.sort_values(by='Movie_box_office_revenue', ascending=False).head(10)

# Average box office revenue by Language
sns.barplot(x=country_revenue.head(10).index, y=country_revenue.head(10)['Movie_box_office_revenue'], ax=axs[1], palette='Greens_r', hue=country_revenue.head(10).index)
axs[1].set_title('Average Box Office Revenue by Country')
axs[1].set_xlabel('Country')
axs[1].set_ylabel('Box Office Revenue (log scale)')
axs[1].set_yscale('log')
axs[1].tick_params(axis='x', rotation=90)  
plt.tight_layout()
plt.show()

## 1.3 Analysis per Genre

In this third part, we delve into the analysis of movies based on their genres. We start by examining the number of movies produced per genre, identifying the most popular genres over time. This is followed by an analysis of the box office revenue and average ratings for each genre, providing insights into the financial success and audience reception of different types of films. By exploring these aspects, we aim to uncover trends and patterns in genre popularity and success, offering a comprehensive understanding of how genre influences the film industry.

In [ ]:
with open(pickle_folder + 'movies_genres_ratings.p', 'rb') as f:
    md_Genres = pickle.load(f)

### 1.3.1 Number of Movies per Genre

The plot of the number of movies per genre highlights that drama, comedy, and romance have a strong foothold in the film industry, while there is also significant production across other genres to meet different audience preferences.

In [ ]:
# Select the top 30 genres
top_genres = md_Genres.value_counts('Genres').head(30)

plt.figure(figsize=(7, 6))
sns.barplot(y=top_genres.index, x=top_genres.values, palette="viridis",hue=top_genres.index)
plt.title('Top Most Frequent Movie Genres')
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.tight_layout()
plt.show()

Analyzing the 10 most frequent Genres in Intervals of 20 Years, we observe that "Drama" has consistently been the dominant genre across all intervals, indicating a stable audience preference for dramatic storytelling over the years. "Comedy" and "Romance Film" also feature prominently in each interval, highlighting their long-standing popularity. However, there are noticeable shifts in other genres; for example, "Black-and-white" films were more common in the earlier intervals (1910-1930 and 1930-1950) but diminish in later years, reflecting the evolution of film technology and color becoming the standard. The presence of genres like "Thriller," "Action," and "World Cinema" increases in the latter half of the 20th century, likely due to globalization and the rise of more diverse storytelling.

In [ ]:
ax = top_n_by_interval('Genres',md_Genres)

### 1.3.2 Box Office Revenue and Average Rating per Genre

Additionally, we analyze the box office revenue and average IMDb ratings for different movie genres to discover insights into the relationship between them, helping us identify trends and patterns.

In [ ]:
genre_stats = md_Genres.groupby('Genres').agg({
    'averageRating': 'mean',
    'numVotes': 'mean',
    'Movie_box_office_revenue': 'mean'
})

fig, axs = plt.subplots(1, 2, figsize=(18, 8))

# Average IMDb rating by Genre (Left plot)
genre_stats = genre_stats.sort_values(by='averageRating', ascending=False)
sns.barplot(x=genre_stats.head(15).index, y=genre_stats.head(15)['averageRating'], ax=axs[0], palette='viridis', hue=genre_stats.head(15).index)
axs[0].set_title("Top 15 Average Rating by Genre")
axs[0].set_xlabel("Genre")
axs[0].set_ylabel("Average Rating")
axs[0].tick_params(axis='x', rotation=90)  

# Average box office revenue by Genre (Right plot)
genre_stats = genre_stats.sort_values(by='Movie_box_office_revenue', ascending=False)
sns.barplot(x=genre_stats.head(15).index, y=genre_stats.head(15)['Movie_box_office_revenue'], ax=axs[1], palette='Blues_r', hue=genre_stats.head(15).index)
axs[1].set_title("Top 15 Average Box Office Revenue by Genre")
axs[1].set_xlabel("Genre")
axs[1].set_ylabel("Average Revenue ($)")
axs[1].set_yscale('log')
axs[1].tick_params(axis='x', rotation=90)  

plt.tight_layout()
plt.show()


In [ ]:
ax = top_n_total_revenue('Genres', md_Genres)

In [ ]:
ax = top_n_by_interval('Genres',md_Genres, revenue=True)

## 1.4 Analysis per Release Season

In this section, we examine the impact of the release season on the number of movies produced and their box office revenues. This analysis provides insights into how the strategic timing, like holidays, of a movie's release can affect its performance at the box office.

In [ ]:
with open(pickle_folder + 'movies_season_ratings.p', 'rb') as f:
    movies_season = pickle.load(f)

### 1.4.1 Number of Movies per Release Season

In [ ]:
plt.figure(figsize=(7, 4))

yearly_counts = movies_season.value_counts('release_season')

ax = sns.barplot(x=yearly_counts.index, y=yearly_counts.values)
plt.title('Total Number of film per season', fontsize=14, pad=15)
plt.xlabel('Season', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)

for i, v in enumerate(yearly_counts.values):
    ax.text(i, v + 1, str(v), 
            ha='center', 
            va='bottom',
            fontsize=10)

plt.tight_layout()

plt.show()

In [ ]:
ax = top_n_by_interval('release_season',movies_season,n=4)

### 1.4.2 Box Office Revenue per Release Season

In [ ]:
release_season_stats = movies_season.groupby('release_season')['Movie_box_office_revenue'].agg(['mean', 'sem']).sort_values(by='mean')
# Print the average revenues by season
print("Average Box Office Revenue by Release Season:")
print(release_season_stats['mean'])

# Plot the average revenues with standard error bars
plt.figure(figsize=(6, 4))
plt.bar(release_season_stats.index, release_season_stats['mean'], yerr=release_season_stats['sem'], color=['orange', 'skyblue', 'lightgreen', 'gold'], capsize=5)
plt.title('Average Box Office Revenue by Release Season with Standard Error')
plt.xlabel('Release Season')
plt.ylabel('Average Revenue ($)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
ax = top_n_by_interval('release_season',movies_season,n=4, revenue=True)

## 1.5. Analysis per Language

In [ ]:
with open(pickle_folder + 'movies_languages_exploded_ratings.p', 'rb') as f:
    movies_languages = pickle.load(f)

In [ ]:
# top 10 languages 
top_languages = movies_languages['Language'].value_counts().head(10)

# Plot 
plt.figure(figsize=(10, 6))
sns.barplot(x=top_languages.index, y=top_languages.values)
plt.title('Top 10 Languages in All Films')
plt.xlabel('Language')
plt.ylabel('Number of Films')
plt.xticks(rotation=90)  
plt.show()

In [ ]:
top_language_data = movies_languages[movies_languages['Language'].isin(top_languages.index)]

language_success = top_language_data.groupby('Language').agg({
    'averageRating': 'mean',
    'numVotes': 'mean',
    'Movie_box_office_revenue': 'mean'
}).sort_values(by='averageRating', ascending=False)

fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# Average imdb rating by Language
sns.barplot(x=language_success.index, y=language_success['averageRating'], ax=axs[0])
axs[0].set_title('Average IMDb Rating by Language')
axs[0].set_xlabel('Language')
axs[0].set_ylabel('Average Rating')
axs[0].tick_params(axis='x', rotation=90)  

# Average box office revenue by Language
sns.barplot(x=language_success.index, y=language_success['Movie_box_office_revenue'], ax=axs[1])
axs[1].set_title('Average Box Office Revenue by Language')
axs[1].set_xlabel('Language')
axs[1].set_ylabel('Box Office Revenue (log scale)')
axs[1].set_yscale('log')
axs[1].tick_params(axis='x', rotation=90)  

plt.tight_layout()
plt.show()

## 1.6. Analysis of the Gender of Actors